In [1]:
import pickle

pkl_filepath = 'data/cache/kitti_train_gt_roidb.pkl'
with open(pkl_filepath, 'rb') as f:
    data = pickle.load(f)

In [6]:
print(len(data))

11968


In [7]:
data[0]

{'boxes': array([[712, 143, 810, 307]], dtype=uint16),
 'gt_classes': array([2], dtype=int32),
 'gt_ishard': array([0], dtype=int32),
 'gt_overlaps': <1x3 sparse matrix of type '<class 'numpy.float32'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'flipped': False,
 'seg_areas': array([16335.], dtype=float32),
 'img_id': 0,
 'image': '/home/superorange5/Research/FedWCD/data/KITTI/JPEGImages/000000.png',
 'width': 1224,
 'height': 370,
 'max_classes': array([2]),
 'max_overlaps': array([1.], dtype=float32)}

In [17]:
data[0].size(0)

AttributeError: 'dict' object has no attribute 'size'

In [ ]:
for datainfo in data:
    wh_pair =[(1242,375),(1224,370),(1238,374),(1241,376)]

    width = datainfo['width']
    height = datainfo['height']
    temp = (width,height)
    if (temp not in wh_pair):
        print(datainfo['image'])
        print (width)
        print (height)

## voc_eval

In [2]:
import os
data_cache_path = 'data/cache'
pkl_file = os.path.join(data_cache_path, 'bdd100k_val' + '_gt_roidb.pkl')

In [23]:
with open(pkl_file, 'rb') as f:
    data = pickle.load(f)

In [3]:
devkit_path = '/work/superorange5/bdd100k'
cachedir = os.path.join(devkit_path, 'annotations_cache')
image_set='val'
imagesetfile = os.path.join(devkit_path,  'ImageSets',
            'Main',   image_set + '.txt')
cachefile = os.path.join(cachedir, '%s_annots.pkl' % imagesetfile)


In [4]:
annopath = os.path.join(devkit_path, 'Annotations', '{:s}.xml')

In [5]:
with open(imagesetfile, 'r') as f:
    lines = f.readlines()
imagenames = [x.strip() for x in lines]

In [6]:
cachefile

'/work/superorange5/bdd100k/ImageSets/Main/val.txt_annots.pkl'

In [7]:
len(imagenames)

10000

In [15]:
import lib.datasets.voc_eval as voc_eval

## generate gt pkl

In [29]:
recs = {}
#for i in range(1000):
#    imagename = imagenames[i]
for i, imagename in enumerate(imagenames):
    recs[imagename] = voc_eval.parse_rec(annopath.format(imagename))
    if i % 100 == 0:
        print('Reading annotation for {:d}/{:d}'.format(
          i + 1, len(imagenames)))
print('Saving cached annotations to {:s}'.format(cachefile))
with open(cachefile, 'wb') as f:
        pickle.dump(recs, f)

Saving cached annotations to /work/superorange5/bdd100k/ImageSets/Main/val.txt_annots.pkl


In [30]:
recs['b1ca8418-84a133a0']

[{'name': 'traffic light', 'truncated': 0, 'bbox': [449, 123, 471, 146]},
 {'name': 'traffic sign', 'truncated': 0, 'bbox': [638, 206, 653, 231]},
 {'name': 'traffic sign', 'truncated': 0, 'bbox': [199, 179, 231, 212]},
 {'name': 'car', 'truncated': 0, 'bbox': [439, 222, 698, 430]},
 {'name': 'car', 'truncated': 1, 'bbox': [1136, 200, 1278, 333]},
 {'name': 'motor', 'truncated': 1, 'bbox': [1204, 232, 1277, 330]}]

## extract gt

In [74]:
import numpy as np
classname = 'car' 
class_recs = {}
npos = 0
ovthresh=0.5
for imagename in imagenames:
    R = [obj for obj in recs[imagename] if obj['name'] == classname]
    bbox = np.array([x['bbox'] for x in R])
    #difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
    det = [False] * len(R)
    npos = npos +  len(R) #sum(~difficult)
    class_recs[imagename] = {'bbox': bbox,
     #                        'difficult': difficult,
                             'det': det}


## read dets

In [75]:
detfile = os.path.join(devkit_path,'results/Main/comp4_det_val_'+classname+'.txt')
with open(detfile, 'r') as f:
    lines = f.readlines()

splitlines = [x.strip().split(' ') for x in lines]
image_ids = [x[0] for x in splitlines]
confidence = np.array([float(x[1]) for x in splitlines])
BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

nd = len(image_ids)
tp = np.zeros(nd)
fp = np.zeros(nd)

if BB.shape[0] > 0:
    # sort by confidence
    sorted_ind = np.argsort(-confidence)
    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    for d in range(nd):
        R = class_recs[image_ids[d]]
        bb = BB[d, :].astype(float)
        ovmax = -np.inf
        BBGT = R['bbox'].astype(float)

        if BBGT.size > 0:
        # compute overlaps
        # intersection
            ixmin = np.maximum(BBGT[:, 0], bb[0])
            iymin = np.maximum(BBGT[:, 1], bb[1])
            ixmax = np.minimum(BBGT[:, 2], bb[2])
            iymax = np.minimum(BBGT[:, 3], bb[3])
            iw = np.maximum(ixmax - ixmin + 1., 0.)
            ih = np.maximum(iymax - iymin + 1., 0.)
            inters = iw * ih

            # union
            uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
                   (BBGT[:, 2] - BBGT[:, 0] + 1.) *
                   (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

            overlaps = inters / uni
            ovmax = np.max(overlaps)
            jmax = np.argmax(overlaps)

        if ovmax > ovthresh:
        #if not R['difficult'][jmax]:
            if not R['det'][jmax]:
                tp[d] = 1.
                R['det'][jmax] = 1
            else:
                fp[d] = 1.
        else:
            fp[d] = 1.

  # compute precision recall
fp = np.cumsum(fp)
tp = np.cumsum(tp)
rec = tp / float(npos)
  # avoid divide by zero in case the first detection matches a difficult
  # ground truth
prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
ap = voc_eval.voc_ap(rec, prec, False)

In [76]:
ap

0.2416113877975409

In [36]:
if not os.path.isfile(cachefile):
    # load annotations
    recs = {}
    for i, imagename in enumerate(imagenames):
      recs[imagename] = parse_rec(annopath.format(imagename))
      if i % 10 == 0:
        print('Reading annotation for {:d}/{:d}'.format(
          i + 1, len(imagenames)))
    # save
    print('Saving cached annotations to {:s}'.format(cachefile))
    with open(cachefile, 'wb') as f:
      pickle.dump(recs, f)

In [26]:

  cachefile = os.path.join(cachedir, '%s_annots.pkl' % imagesetfile)
  # read list of images
  with open(imagesetfile, 'r') as f:
    lines = f.readlines()
  imagenames = [x.strip() for x in lines]

  if not os.path.isfile(cachefile):
    # load annotations
    recs = {}
    for i, imagename in enumerate(imagenames):
      recs[imagename] = parse_rec(annopath.format(imagename))
      if i % 100 == 0:
        print('Reading annotation for {:d}/{:d}'.format(
          i + 1, len(imagenames)))
    # save
    print('Saving cached annotations to {:s}'.format(cachefile))
    with open(cachefile, 'wb') as f:
      pickle.dump(recs, f)
  else:
    # load
    with open(cachefile, 'rb') as f:
      try:
        recs = pickle.load(f)
      except:
        recs = pickle.load(f, encoding='bytes')

  # extract gt objects for this class
  class_recs = {}
  npos = 0
  for imagename in imagenames:
    R = [obj for obj in recs[imagename] if obj['name'] == classname]
    bbox = np.array([x['bbox'] for x in R])
    #difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
    det = [False] * len(R)
    #npos = npos + sum(~difficult)
    class_recs[imagename] = {'bbox': bbox,
     #                        'difficult': difficult,
                             'det': det}

  # read dets
  detfile = detpath.format(classname)
  with open(detfile, 'r') as f:
    lines = f.readlines()

  splitlines = [x.strip().split(' ') for x in lines]
  image_ids = [x[0] for x in splitlines]
  confidence = np.array([float(x[1]) for x in splitlines])
  BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

  nd = len(image_ids)
  tp = np.zeros(nd)
  fp = np.zeros(nd)

  if BB.shape[0] > 0:
    # sort by confidence
    sorted_ind = np.argsort(-confidence)
    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    for d in range(nd):
      R = class_recs[image_ids[d]]
      bb = BB[d, :].astype(float)
      ovmax = -np.inf
      BBGT = R['bbox'].astype(float)

      if BBGT.size > 0:
        # compute overlaps
        # intersection
        ixmin = np.maximum(BBGT[:, 0], bb[0])
        iymin = np.maximum(BBGT[:, 1], bb[1])
        ixmax = np.minimum(BBGT[:, 2], bb[2])
        iymax = np.minimum(BBGT[:, 3], bb[3])
        iw = np.maximum(ixmax - ixmin + 1., 0.)
        ih = np.maximum(iymax - iymin + 1., 0.)
        inters = iw * ih

        # union
        uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
               (BBGT[:, 2] - BBGT[:, 0] + 1.) *
               (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

        overlaps = inters / uni
        ovmax = np.max(overlaps)
        jmax = np.argmax(overlaps)

      if ovmax > ovthresh:
        #if not R['difficult'][jmax]:
        if not R['det'][jmax]:
          tp[d] = 1.
          R['det'][jmax] = 1
        else:
          fp[d] = 1.
      else:
        fp[d] = 1.

  # compute precision recall
  fp = np.cumsum(fp)
  tp = np.cumsum(tp)
  rec = tp / float(npos)
  # avoid divide by zero in case the first detection matches a difficult
  # ground truth
  prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
  ap = voc_ap(rec, prec, use_07_metric)

  return rec, prec, ap

{'boxes': array([[1001,  282, 1041,  327],
        [ 215,  172,  275,  230],
        [ 797,  313,  830,  342],
        [ 653,  303,  685,  316],
        [ 707,  312,  716,  328],
        [ 626,  296,  636,  317],
        [ 317,  289,  329,  307],
        [ 271,  288,  309,  302],
        [ 221,  301,  232,  312],
        [ 206,  338,  282,  388],
        [  47,  344,  130,  401],
        [ 247,  344,  347,  396],
        [   0,  337,   52,  403],
        [ 650,  354,  664,  382],
        [ 649,  367,  664,  388],
        [ 712,  337,  726,  353],
        [ 684,  357,  721,  393],
        [ 706,  364,  734,  391],
        [ 727,  366,  761,  401],
        [ 749,  362,  809,  409],
        [ 787,  358,  906,  425],
        [ 880,  376,  960,  452],
        [ 936,  336, 1212,  484],
        [1204,  416, 1279,  530],
        [ 603,  341,  610,  353],
        [ 591,  328,  598,  342],
        [ 529,  332,  537,  342],
        [ 552,  356,  565,  368],
        [ 574,  352,  586,  369],
     

## bg_num_rois = 0 and fg_num_rois = 0, this should not happen!

In [84]:
import pickle

pkl_filepath = 'data/cache/multi_ck_train_gt_roidb.pkl'
with open(pkl_filepath, 'rb') as f:
    multick_data = pickle.load(f)

In [87]:
len(multick_data)

17898

In [86]:
pkl_filepath = 'data/cache/kitti_train_gt_roidb.pkl'
with open(pkl_filepath, 'rb') as f:
    kiti_data = pickle.load(f)

In [88]:
len(kiti_data)

11968

In [112]:
kiti_data[5984]

{'boxes': array([[414, 143, 512, 307]], dtype=uint16),
 'gt_overlaps': <1x3 sparse matrix of type '<class 'numpy.float32'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'gt_classes': array([2], dtype=int32),
 'flipped': True,
 'img_id': 5984,
 'image': '/home/superorange5/Research/FedWCD/data/KITTI/JPEGImages/000000.png',
 'width': 1224,
 'height': 370,
 'max_classes': array([2]),
 'max_overlaps': array([1.], dtype=float32)}

In [111]:
multick_data[11914]

{'boxes': array([[414, 143, 512, 307]], dtype=uint16),
 'gt_overlaps': <1x9 sparse matrix of type '<class 'numpy.float32'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'gt_classes': array([2], dtype=int32),
 'flipped': True,
 'img_id': 11914,
 'image': '/home/superorange5/Research/FedWCD/data/KITTI/JPEGImages/000000.png',
 'width': 1224,
 'height': 370,
 'max_classes': array([2]),
 'max_overlaps': array([1.], dtype=float32)}

## roidb rank width error

In [126]:
for i in range(len(kiti_data)):
  #    print('#####'+str(i))
    width = multick_data[i]['width']
    #print(width)

## create a new

In [119]:
new_multick = multick_data[0:100]+multick_data[2965:3065]

In [120]:
len(new_multick)

200

In [121]:
cache_file = 'data/cache/multi_ck_trainsmall'
with open(cache_file, 'wb') as fid:
    pickle.dump(new_multick, fid)

In [82]:
for data_info in data:
    bboxes = data_info['boxes']
    if len(bboxes) <2:
        print (data_info)

{'boxes': array([[794, 430, 875, 487]], dtype=uint16), 'gt_classes': array([1], dtype=int32), 'gt_ishard': array([0], dtype=int32), 'gt_overlaps': <1x9 sparse matrix of type '<class 'numpy.float32'>'
	with 1 stored elements in Compressed Sparse Row format>, 'flipped': False, 'seg_areas': array([4756.], dtype=float32), 'img_id': 343, 'image': '/home/superorange5/Research/FedWCD/data/cityscape/VOC2007/JPEGImages/strasbourg_000001_043748_leftImg8bit.png', 'width': 2048, 'height': 1024, 'max_classes': array([1]), 'max_overlaps': array([1.], dtype=float32)}
{'boxes': array([[1335,  277, 2048,  637]], dtype=uint16), 'gt_classes': array([1], dtype=int32), 'gt_ishard': array([0], dtype=int32), 'gt_overlaps': <1x9 sparse matrix of type '<class 'numpy.float32'>'
	with 1 stored elements in Compressed Sparse Row format>, 'flipped': False, 'seg_areas': array([257754.], dtype=float32), 'img_id': 621, 'image': '/home/superorange5/Research/FedWCD/data/cityscape/VOC2007/JPEGImages/tubingen_000136_00001